Topic: Project 3      
Subject: Cleaning League of Legends Data   
Date: 10/25/2017   
Name: Zach Heick  

**Order of Events**  
  1. [Storing Raw League of Legends Data](https://github.com/ZachHeick/Project_McNulty/blob/master/Project_Notebooks/Project_McNulty_SQL_Alchemy.ipynb)  
  2. [Cleaning League of Legends Data](https://github.com/ZachHeick/Project_McNulty/blob/master/Project_Notebooks/Project_McNulty_Cleaning.ipynb)  
  3. [Creating Dataframes for EDA](https://github.com/ZachHeick/Project_McNulty/blob/master/Project_Notebooks/Project_McNulty_Creating_EDA_Dataframes.ipynb)  
  4. [Visualizing League of Legends Data](https://github.com/ZachHeick/Project_McNulty/blob/master/Project_Notebooks/Project_McNulty_EDA_Visuals.ipynb)  
  5. [Analyzing Classification Algorithms](https://github.com/ZachHeick/Project_McNulty/blob/master/Project_Notebooks/Project_McNulty_Modeling.ipynb)

**Summary**: Raw data for match history, champions, and summoner spells are pulled down from a Postgres database, cleaned, and pickled into dataframes for data analysis, visualization, and modeling. 

In [1]:
import pickle
import pandas as pd
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os

%matplotlib inline

Connect to Postgres database.

In [4]:
engine_name = 'postgresql://' + str(os.environ['zU']) + ':' + str(os.environ['zP']) + '@13.59.55.238:5432/project_mcnulty'
cnx = create_engine(engine_name)

# Match History

Load match history data from Postgres database. 

In [5]:
raw_games_df = pd.read_sql('games', cnx)
raw_games_df.drop('index', axis=1, inplace=True)

A look at all of the columns and the dataframe. The target column is `winner`, where the value is either team 1 or team 2. Columns that begin with `first` represent which team was first to secure this objective, where 1 and 2 represent the team and 0 represents neither. The rest of the columns are the ID's of each player's champion, summoner spells, and their ban. Before the game starts, each player bans away a champion from the other team before they pick their own.

In [43]:
raw_games_df.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


In [44]:
raw_games_df.columns

Index(['gameId', 'creationTime', 'gameDuration', 'seasonId', 'winner',
       'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron',
       'firstDragon', 'firstRiftHerald', 't1_champ1id', 't1_champ1_sum1',
       't1_champ1_sum2', 't1_champ2id', 't1_champ2_sum1', 't1_champ2_sum2',
       't1_champ3id', 't1_champ3_sum1', 't1_champ3_sum2', 't1_champ4id',
       't1_champ4_sum1', 't1_champ4_sum2', 't1_champ5id', 't1_champ5_sum1',
       't1_champ5_sum2', 't1_towerKills', 't1_inhibitorKills', 't1_baronKills',
       't1_dragonKills', 't1_riftHeraldKills', 't1_ban1', 't1_ban2', 't1_ban3',
       't1_ban4', 't1_ban5', 't2_champ1id', 't2_champ1_sum1', 't2_champ1_sum2',
       't2_champ2id', 't2_champ2_sum1', 't2_champ2_sum2', 't2_champ3id',
       't2_champ3_sum1', 't2_champ3_sum2', 't2_champ4id', 't2_champ4_sum1',
       't2_champ4_sum2', 't2_champ5id', 't2_champ5_sum1', 't2_champ5_sum2',
       't2_towerKills', 't2_inhibitorKills', 't2_baronKills', 't2_dragonKills',
       't2_riftHer

I drop columns that I won't need for analysis or modeling. I'll treat inhibitors and the nexus as a single structure and also drop `t1_inhibitorKills` and `t2_inhibitorKills`. Because of how close inhibitors are to the nexus, teams usually destroy the inhibitors in sequence with the nexus. This means that if a team has more inhibitor kills, there is a good chance that they destroyed the nexus right after, and are therefore the winner. When predicting a winner, I want to focus more on team composition and what structures or neutral objectives teams took first.

In [45]:
raw_games_df.drop(['gameId', 'creationTime', 'seasonId', 't1_inhibitorKills', 't2_inhibitorKills'], axis=1, inplace=True)

# Champions

Load champion data and drop extra index column.

In [125]:
champ_info_df = pd.read_sql('champions', cnx)
champ_info_df.drop('index', axis=1, inplace=True)
champ_info_df.head()

,type,version,data__|,data__|__tags__001,data__|__tags__002,data__|__title,data__|__id,data__|__key,data__|__name
0,champion,7.18.1,None,None,None,None,-1,None,None
1,None,None,MonkeyKing,Fighter,Tank,the Monkey King,62,MonkeyKing,Wukong
2,None,None,Jax,Fighter,Assassin,Grandmaster at Arms,24,Jax,Jax
3,None,None,Fiddlesticks,Mage,Support,the Harbinger of Doom,9,Fiddlesticks,Fiddlesticks
4,None,None,Shaco,Assassin,None,the Demon Jester,35,Shaco,Shaco


In [126]:
champ_info_df.columns

Index(['type', 'version', 'data__|', 'data__|__tags__001',
       'data__|__tags__002', 'data__|__title', 'data__|__id', 'data__|__key',
       'data__|__name'],
      dtype='object')

The column names are not nice to read right now, but I'll drop the ones I don't need. The columns I do need are the primary role of the champion, their game ID number, and their name.

In [127]:
champ_info_df.drop([
    'type',
    'version',
    'data__|',
    'data__|__tags__002',
    'data__|__title',
    'data__|__key',
], axis=1, inplace=True)

In [128]:
champ_info_df.head()

,data__|__tags__001,data__|__id,data__|__name
0,None,-1,None
1,Fighter,62,Wukong
2,Fighter,24,Jax
3,Mage,9,Fiddlesticks
4,Assassin,35,Shaco


I'll rename my columns and finish cleaning up this dataframe. I'll also change the index to the champion's game ID.

In [129]:
champ_info_df.columns = ['role', 'id', 'name']
champ_info_df['role'] = champ_info_df['role'][1:].apply(lambda i: i.lower())
champ_info_df.set_index(['id'], inplace=True)
champ_info_df['role'].fillna('None', inplace=True)
champ_info_df.head()

,role,name
id,,
-1,None,None
62,fighter,Wukong
24,fighter,Jax
9,mage,Fiddlesticks
35,assassin,Shaco


From playing the game, I know that there are some champs whose roles are misclassified, so I'll fix those.

In [131]:
champ_info_df.loc[champ_info_df['name'] == 'Morgana', 'role'] = 'support'
champ_info_df.loc[champ_info_df['name'] == 'Leona', 'role'] = 'support'
champ_info_df.loc[champ_info_df['name'] == 'Blitzcrank', 'role'] = 'support'
champ_info_df.loc[champ_info_df['name'] == 'Ivern', 'role'] = 'tank'
champ_info_df.loc[champ_info_df['name'] == 'Nunu', 'role'] = 'tank'

# Summoner Spells

Load summoner spell data and drop extra index column.

In [151]:
summ_spell_df = pd.read_sql('summoner_spells', cnx)
summ_spell_df.drop('index', axis=1, inplace=True)
summ_spell_df.head()

,type,version,data__|,data__|__id,data__|__summonerLevel,data__|__name,data__|__key,data__|__description
0,summoner,7.17.2,1,1,6,Cleanse,SummonerBoost,Removes all disables (excluding suppression an...
1,None,None,3,3,4,Exhaust,SummonerExhaust,"Exhausts target enemy champion, reducing their..."
2,None,None,4,4,8,Flash,SummonerFlash,Teleports your champion a short distance towar...
3,None,None,6,6,1,Ghost,SummonerHaste,Your champion gains increased Movement Speed a...
4,None,None,7,7,1,Heal,SummonerHeal,Restores 90-345 Health (depending on champion ...


In [152]:
summ_spell_df.columns

Index(['type', 'version', 'data__|', 'data__|__id', 'data__|__summonerLevel',
       'data__|__name', 'data__|__key', 'data__|__description'],
      dtype='object')

Similar to the raw champion dataset, the columns that I don't want are dropped. The columns that I do want are the game ID number, and the name of the summoner spell.

In [153]:
summ_spell_df.drop([
    'type',
    'version',
    'data__|',
    'data__|__summonerLevel',
    'data__|__key',
    'data__|__description'
], axis=1, inplace=True)

In [154]:
summ_spell_df.head()

,data__|__id,data__|__name
0,1,Cleanse
1,3,Exhaust
2,4,Flash
3,6,Ghost
4,7,Heal


Columns are renamed and the dataframe for summoner spells is now clean.

In [138]:
summ_spell_df.columns = ['id', 'name']
summ_spell_df['name'] = summ_spell_df['name'].apply(lambda i: i.lower())
summ_spell_df.set_index(['id'], inplace=True)
summ_spell_df.head()

,name
id,
1,cleanse
3,exhaust
4,flash
6,ghost
7,heal


# Merging Datasets and Cleaning

I'll change columns that begin with `t1` to `blue` and `t2` to `red` in `raw_games_df`.

In [140]:
new_col_names = ['gameDuration','winner','firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald']
for col_name in raw_games_df.columns[8:]:
    if col_name[:2] == 't1':
        new_col_name = 'blue' + col_name[2:]
    else:
        new_col_name = 'red' + col_name[2:]
    new_col_names.append(new_col_name)

In [141]:
raw_games_df.columns = new_col_names
raw_games_df.columns

Index(['gameDuration', 'winner', 'firstBlood', 'firstTower', 'firstInhibitor',
       'firstBaron', 'firstDragon', 'firstRiftHerald', 'blue_champ1id',
       'blue_champ1_sum1', 'blue_champ1_sum2', 'blue_champ2id',
       'blue_champ2_sum1', 'blue_champ2_sum2', 'blue_champ3id',
       'blue_champ3_sum1', 'blue_champ3_sum2', 'blue_champ4id',
       'blue_champ4_sum1', 'blue_champ4_sum2', 'blue_champ5id',
       'blue_champ5_sum1', 'blue_champ5_sum2', 'blue_towerKills',
       'blue_baronKills', 'blue_dragonKills', 'blue_riftHeraldKills',
       'blue_ban1', 'blue_ban2', 'blue_ban3', 'blue_ban4', 'blue_ban5',
       'red_champ1id', 'red_champ1_sum1', 'red_champ1_sum2', 'red_champ2id',
       'red_champ2_sum1', 'red_champ2_sum2', 'red_champ3id', 'red_champ3_sum1',
       'red_champ3_sum2', 'red_champ4id', 'red_champ4_sum1', 'red_champ4_sum2',
       'red_champ5id', 'red_champ5_sum1', 'red_champ5_sum2', 'red_towerKills',
       'red_baronKills', 'red_dragonKills', 'red_riftHeraldKills', 'r

In [142]:
champ_cols = [
    'blue_champ1id',
    'blue_champ2id',
    'blue_champ3id',
    'blue_champ4id',
    'blue_champ5id',
    'red_champ1id',
    'red_champ2id',
    'red_champ3id',
    'red_champ4id',
    'red_champ5id'
]

summ_spell_cols = [
    'blue_champ1_sum1',
    'blue_champ1_sum2',
    'blue_champ2_sum1',
    'blue_champ2_sum2',
    'blue_champ3_sum1',
    'blue_champ3_sum2',
    'blue_champ4_sum1',
    'blue_champ4_sum2',
    'blue_champ5_sum1',
    'blue_champ5_sum2',
    'red_champ1_sum1',
    'red_champ1_sum2',
    'red_champ2_sum1',
    'red_champ2_sum2',
    'red_champ3_sum1',
    'red_champ3_sum2',
    'red_champ4_sum1',
    'red_champ4_sum2',
    'red_champ5_sum1',
    'red_champ5_sum2',
]

ban_cols = [
    'blue_ban1',
    'blue_ban2',
    'blue_ban3',
    'blue_ban4',
    'blue_ban5',
    'red_ban1',
    'red_ban2',
    'red_ban3',
    'red_ban4',
    'red_ban5'
]

When banning champions away from the other team, there is an option to not ban anyone. I only want to look at games where each team had five bans, so I'll drop rows where this is not the case.

In [143]:
bans_df = raw_games_df[ban_cols]
ban_indexes = bans_df[bans_df.values == -1].index.tolist()

In [144]:
raw_games_df.drop(raw_games_df.index[ban_indexes], inplace=True)
raw_games_df.reset_index(inplace=True, drop=True)

In [145]:
raw_games_df.head(10)

,gameDuration,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,blue_champ1id,blue_champ1_sum1,...,red_champ5_sum2,red_towerKills,red_baronKills,red_dragonKills,red_riftHeraldKills,red_ban1,red_ban2,red_ban3,red_ban4,red_ban5
0,1949,1,2,1,1,1,1,2,8,12,...,3,5,0,1,1,114,67,43,16,51
1,1851,1,1,1,1,0,1,1,119,7,...,7,2,0,0,0,11,67,238,51,420
2,1493,1,2,1,1,1,2,0,18,4,...,4,2,0,1,0,157,238,121,57,28
3,1758,1,1,1,1,1,1,0,57,4,...,7,0,0,0,0,164,18,141,40,51
4,2094,1,2,1,1,1,1,0,19,4,...,7,3,0,1,0,86,11,201,122,18
5,2059,1,2,2,1,1,2,0,40,3,...,14,6,0,3,0,119,134,154,63,31
6,1993,1,1,2,1,1,1,1,74,3,...,4,2,0,0,0,75,42,31,40,429
7,1334,1,1,1,0,0,2,1,150,12,...,12,0,0,2,0,157,38,122,40,238
8,1387,2,2,2,2,0,2,2,111,12,...,4,8,0,2,1,28,51,53,157,40
9,2681,2,2,2,2,2,2,0,427,4,...,4,8,1,2,0,64,141,84,67,53


Pickle this dataframe for modeling.

In [146]:
raw_games_df.to_pickle('model_df.pickle')

In [147]:
def id_to_value(df, id_, col):
    """
    Takes an ID located in a cell and replaces with its value
    :param df: the joined dataframe
    :id_: the numeric ID
    :col: the column of the dataframe
    :return: the matching value
    """
    return df.loc[id_][col]

I want to have another dataframe for data analysis and visualization. I'll iterate through all columns containing IDs to champions and summoner spells and replace them with the actual name.

In [148]:
for col in champ_cols+ban_cols:
    raw_games_df[col] = raw_games_df[col].apply(lambda i: id_to_value(champ_info_df, i, 'name'))

for col in summ_spell_cols:
    raw_games_df[col] = raw_games_df[col].apply(lambda i: id_to_value(summ_spell_df, i, 'name'))

In [149]:
raw_games_df.head(10)

,gameDuration,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,blue_champ1id,blue_champ1_sum1,...,red_champ5_sum2,red_towerKills,red_baronKills,red_dragonKills,red_riftHeraldKills,red_ban1,red_ban2,red_ban3,red_ban4,red_ban5
0,1949,1,2,1,1,1,1,2,Vladimir,teleport,...,exhaust,5,0,1,1,Fiora,Vayne,Karma,Soraka,Caitlyn
1,1851,1,1,1,1,0,1,1,Draven,heal,...,heal,2,0,0,0,Master Yi,Vayne,Zed,Caitlyn,Illaoi
2,1493,1,2,1,1,1,2,0,Tristana,flash,...,flash,2,0,1,0,Yasuo,Zed,Kha'Zix,Maokai,Evelynn
3,1758,1,1,1,1,1,1,0,Maokai,flash,...,heal,0,0,0,0,Camille,Tristana,Kayn,Janna,Caitlyn
4,2094,1,2,1,1,1,1,0,Warwick,flash,...,heal,3,0,1,0,Garen,Master Yi,Braum,Darius,Tristana
5,2059,1,2,2,1,1,2,0,Janna,exhaust,...,ignite,6,0,3,0,Draven,Syndra,Zac,Brand,Cho'Gath
6,1993,1,1,2,1,1,1,1,Heimerdinger,exhaust,...,flash,2,0,0,0,Nasus,Corki,Cho'Gath,Janna,Kalista
7,1334,1,1,1,0,0,2,1,Gnar,teleport,...,teleport,0,0,2,0,Yasuo,Kassadin,Darius,Janna,Zed
8,1387,2,2,2,2,0,2,2,Nautilus,teleport,...,flash,8,0,2,1,Evelynn,Caitlyn,Blitzcrank,Yasuo,Janna
9,2681,2,2,2,2,2,2,0,Ivern,flash,...,flash,8,1,2,0,Lee Sin,Kayn,Akali,Vayne,Blitzcrank


Pickle the games, champions, and summoner spells dataframes for EDA.

In [150]:
raw_games_df.to_pickle('EDA_df.pickle')
champ_info_df.to_pickle('champ_info.pickle')
summ_spell_df.to_pick('summ_spell_info.pickle')